<a href="https://colab.research.google.com/github/achillesposiedon/TaxiGameRL/blob/master/CartPoleDQN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [51]:
%tensorflow_version 1.x

TensorFlow is already loaded. Please restart the runtime to change versions.


In [0]:
import numpy as np
import gym
import random

from keras.models import Sequential    
from keras.layers import Dense, Flatten
from collections import deque  
import matplotlib.pyplot as plt

from IPython import display as ipythondisplay

In [0]:
env = gym.make('CartPole-v1') 

In [73]:
model=Sequential()

model.add(Dense(20, input_shape=(2,) + env.observation_space.shape, init='uniform', activation='relu'))
model.add(Flatten())    
model.add(Dense(18, init='uniform', activation='relu'))
model.add(Dense(10, init='uniform', activation='relu'))
model.add(Dense(env.action_space.n, init='uniform', activation='linear'))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(20, input_shape=(2, 4), activation="relu", kernel_initializer="uniform")`
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(18, activation="relu", kernel_initializer="uniform")`
  """
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(10, activation="relu", kernel_initializer="uniform")`
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(2, activation="linear", kernel_initializer="uniform")`
  import sys


In [0]:
model.compile(loss='mse', optimizer='adam', metrics=['accuracy'])

In [0]:
D=deque() #double queue 
time=5000
epsilon=0.5
gamma=0.9
minibatchsize=500

In [62]:
trail = env.reset()                     
st1 = np.expand_dims(trail, axis=0)      
state = np.stack((st1,st1), axis=1) # 2 times because we are using 2 consequtive states of the game for learning
done = False
for t in range(time):
    if np.random.rand() <= epsilon: #Explore: take random action 60 percent of the time
        action = np.random.randint(0, env.action_space.n, size=1)[0]
    else:
        Q = model.predict(state)          # Q-value predictions from Neural net
        action = np.argmax(Q)             
    st2=state[:,1]
    
    newstate, reward, done, info = env.step(action)     
    newstate = np.expand_dims(newstate, axis=0)          
    state_new = np.stack((newstate,st2), axis=1)
    D.append((state, action, reward, state_new, done))         # 'Remember' action and consequence
    state = state_new         # Update state
    if done:
        env.reset()           # Restart game if it's finished
        obs = np.expand_dims(trail, axis=0)     # (Formatting issues) Making the observation the first element of a batch of inputs 
        state = np.stack((st1, st1), axis=1)
print('Exploring Done')


Exploring Done


In [0]:
# Experience replay

minibatch = random.sample(D, minibatchsize)                              # Sample some random moves from exploration memory
for state, action, reward, state_next, terminal in minibatch:
        q_update = reward
        if not terminal:
            q_update = (reward + gamma * np.amax(model.predict(state_next)[0]))
        q_values = model.predict(state)
        q_values[0][action] = q_update
        model.fit(state, q_values, verbose=0)

In [64]:
#Play!

observation = env.reset()
rlst = np.expand_dims(observation, axis=0)
state = np.stack((rlst, rlst), axis=1)
done = False
finalreward = 0.0
while not done:
    #env.render()   
    st2=state[:,1]
                    
    Q = model.predict(state)        
    action = np.argmax(Q)         
    observation, reward, done, info = env.step(action)
    obs = np.expand_dims(observation, axis=0)
    state = np.stack((newstate,st2), axis=1)    
    finalreward += reward
print('Game ended! Total reward: {}'.format(finalreward))

Game ended! Total reward: 10.0


In [0]:
for i in range(1000):
  trail = env.reset()                     
  st1 = np.expand_dims(trail, axis=0)      
  state = np.stack((st1,st1), axis=1) # 2 times because we are using 2 consequtive states of the game for learning
  done = False
  for t in range(time):
    if np.random.rand() <= epsilon: #Explore: take random action 60 percent of the time
        action = np.random.randint(0, env.action_space.n, size=1)[0]
    else:
        Q = model.predict(state)          # Q-value predictions from Neural net
        action = np.argmax(Q)             
    st2=state[:,1]
    newstate, reward, done, info = env.step(action)     
    newstate = np.expand_dims(newstate, axis=0)          
    state_new = np.stack((newstate,st2), axis=1)
    D.append((state, action, reward, state_new, done))         # 'Remember' action and consequence
    state = state_new         # Update state
    if done:
        env.reset()           # Restart game if it's finished
        obs = np.expand_dims(trail, axis=0)     # (Formatting issues) Making the observation the first element of a batch of inputs 
        state = np.stack((st1, st1), axis=1)


  minibatch = random.sample(D, minibatchsize)                              # Sample some random moves from exploration memory
  for state, action, reward, state_next, terminal in minibatch:
    q_update = reward
    if not terminal:
      q_update = (reward + gamma * np.amax(model.predict(state_next)[0]))
    q_values = model.predict(state)
    q_values[0][action] = q_update
    model.fit(state, q_values, verbose=0)

  observation = env.reset()
  rlst = np.expand_dims(observation, axis=0)
  state3 = np.stack((rlst, rlst), axis=1)
  done = False
  finalreward3 = 0.0
  while not done:
    #env.render()   
    st23=state3[:,1]
                    
    Q3 = model.predict(state3)        
    action3 = np.argmax(Q3)         
    observation3, reward3, done, info = env.step(action3)
    obs3 = np.expand_dims(observation3, axis=0)
    state3 = np.stack((obs3,st2), axis=1)    
    finalreward3 += reward3
  print(finalreward3)
    


9.0
8.0
9.0
8.0
8.0
10.0
10.0
10.0
9.0
9.0
10.0
9.0
8.0
10.0
10.0
9.0
10.0
9.0
8.0
9.0
10.0
10.0
10.0
10.0
10.0
9.0
9.0
11.0
9.0
10.0
10.0
9.0
8.0
10.0
9.0
11.0
8.0
10.0
9.0
15.0
10.0
8.0
10.0
10.0
9.0
9.0
8.0
26.0
10.0
10.0
11.0
10.0
12.0
10.0
9.0
13.0
18.0
11.0
10.0
10.0
61.0
11.0
10.0
8.0
13.0
10.0
22.0
12.0
52.0
17.0
10.0
10.0
31.0
9.0
10.0
10.0
16.0
10.0
13.0
73.0
10.0
10.0
9.0
10.0
14.0
9.0
13.0
9.0
8.0
27.0
167.0
23.0
13.0
9.0
19.0
139.0
14.0
12.0
21.0
359.0
13.0
97.0
227.0
100.0
355.0
107.0
21.0
109.0
50.0
84.0
122.0
120.0
93.0
500.0
126.0
500.0
500.0
90.0
95.0
134.0
26.0
241.0
13.0
500.0
128.0
135.0
122.0
293.0
139.0
179.0
459.0
186.0
181.0
149.0
128.0
149.0
